# Рекомендация тарифов

Проектная работа

Описание проекта
Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».
В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.
Постройте модель с максимально большим значением accuracy. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте accuracy на тестовой выборке самостоятельно.

In [55]:
import math

import pandas as pd
import seaborn as sns
import scipy.stats as st
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sys import getsizeof


try:
    from IPython.core.display_functions import display
except ModuleNotFoundError:
    pass


In [29]:
%config InlineBackend.figure_formats = ['svg']

In [30]:
# seaborn style via MATPLOTLIBRC
custom_params = {'figure.figsize': (10, 6),
                'figure.facecolor': '#232425',
                'figure.dpi': 240,

                'legend.frameon': False,
                'legend.borderpad': 1.4,
                'legend.labelspacing': 0.7,
                'legend.handlelength': 0.7,
                'legend.handleheight': 0.7,

                'axes.facecolor': '#232425',
                'axes.labelcolor': '#EEEEEE',
                'axes.labelpad': 17,
                'axes.spines.left': False,
                'axes.spines.bottom': False,
                'axes.spines.right': False,
                'axes.spines.top': False,
                'axes.grid': False,

                'contour.linewidth': 0.0,

                'xtick.color': '#AAAAAA',
                'ytick.color': '#AAAAAA',
                'xtick.bottom': True,
                'xtick.top': False,
                'ytick.left': True,
                'ytick.right': False,

                'text.color': '#EEEEEE',
                'font.sans-serif': [
                    'Helvetica',
                    'Verdana',
                    'sans-serif'
                    ]
                }
sns.set_theme(style='darkgrid', rc=custom_params)
sns.set_context("notebook", font_scale=1.37)

## Откройте файл с данными и изучите его.

In [31]:
try:
    accu_df = pd.read_csv('./datasets/users_behavior.csv')
except FileNotFoundError:
    accu_df = pd.read_csv('https://code.s3.yandex.net/datasets/users_behavior.csv')
    print('FYI csv loaded by url')

In [32]:
display(
    accu_df.sample(5)
)

calls  minutes  messages   mb_used  is_ultra
1884   60.0   407.95       0.0  26965.37         0
3103   63.0   435.98      32.0  18295.22         0
930   103.0   730.61     154.0  18565.13         1
26     81.0   624.71      38.0   8644.66         1
1834   62.0   448.76      62.0  15522.01         0

In [33]:
display(
    accu_df.describe()
)

calls      minutes     messages       mb_used     is_ultra
count  3214.000000  3214.000000  3214.000000   3214.000000  3214.000000
mean     63.038892   438.208787    38.281269  17207.673836     0.306472
std      33.236368   234.569872    36.148326   7570.968246     0.461100
min       0.000000     0.000000     0.000000      0.000000     0.000000
25%      40.000000   274.575000     9.000000  12491.902500     0.000000
50%      62.000000   430.600000    30.000000  16943.235000     0.000000
75%      82.000000   571.927500    57.000000  21424.700000     1.000000
max     244.000000  1632.060000   224.000000  49745.730000     1.000000

In [42]:
display(
    accu_df.info()
)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   uint8  
 1   minutes   3214 non-null   float32
 2   messages  3214 non-null   uint8  
 3   mb_used   3214 non-null   float32
 4   is_ultra  3214 non-null   bool   
dtypes: bool(1), float32(2), uint8(2)
memory usage: 34.7 KB


None

In [43]:
accu_df = accu_df.astype(
    {
        'calls': 'uint8',
        'minutes': 'float32',
        'messages': 'uint8',
        'mb_used': 'float32',
        'is_ultra': 'bool',
    }
)

In [44]:
print(f'Размер набора данных после замены типов данных:',
    round((getsizeof(accu_df) / 1024), 2)
    , 'KB'
)

Размер набора данных после замены типов данных: 34.67 KB


In [47]:
features = accu_df.drop(['is_ultra'], axis=1)
target = accu_df['is_ultra']


## Разделите исходные данные на обучающую, валидационную и тестовую выборки.

> Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.


Используем стандартные пропорции для деления набора данных:
- обучающая выборка 60%
- валидационная выборка 20%
- тестовая выборка 20%


In [48]:
features_train, features_valid,\
target_train, target_valid = train_test_split(
    features, target,
    test_size=0.2,
    shuffle = True,
    random_state=108
)

features_train, features_test,\
target_train, target_test = train_test_split(
    features_train,
    target_train,
    test_size=0.25,
    random_state=108
)

In [54]:
print('x_train', features_train.shape)
print('x_valid', features_valid.shape)
print('x_test', features_test.shape)
print()
print('y_train', target_train.shape)
print('y_valid', target_valid.shape)
print('y_test', target_test.shape)


x_train (1928, 4)
x_valid (643, 4)
x_test (643, 4)

y_train (1928,)
y_valid (643,)
y_test (643,)


Соотношения выборок похожи на правду 👏

## Исследуйте качество разных моделей, меняя гиперпараметры.

In [ ]:
best_model = None
best_result = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(
        random_state=12345,
        max_depth=depth)
    model.fit(features, target)
    predictions = model.predict(features) # получите предсказания модели
    result = accuracy_score(target, predictions) # посчитайте качество модели
    if result > best_result:
        best_model = model
        best_result = result

print("Accuracy лучшей модели:", best_result)

### Кратко напишите выводы исследования.

## Проверьте качество модели на тестовой выборке.

## Дополнительное задание: проверьте модели на вменяемость.

## Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. Известно:

`сalls` — количество звонков,
`minutes` — суммарная длительность звонков в минутах,
`messages` — количество sms-сообщений,
`mb_used` — израсходованный интернет-трафик в Мб,
`is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Как будут проверять мой проект?
Мы подготовили критерии оценки проекта, которыми руководствуются ревьюеры. Прежде чем приступить к решению кейса, внимательно их изучите.
На что обращают внимание ревьюеры, проверяя проект:

- Как вы изучаете данные после загрузки?
- Корректно ли разделяете данные на выборки?
- Как выбираете размеры выборок?
- Правильно ли вы оцениваете качество моделей в исследовании?
- Какие модели и гиперпараметры вы используете?
- Какие выводы об исследовании делаете?
- Правильно ли тестируете модели?
- Насколько высокое значение accuracy получаете?
- Соблюдаете структуру проекта и поддерживаете аккуратность кода?